In [1133]:
!which python 
!whoami 
!hostname
!pwd

/home/farshed.abdukhakimov/.conda/envs/psps/bin/python
farshed.abdukhakimov
cn-08
/home/farshed.abdukhakimov/projects/psps2


In [1134]:
import os
import sys
import urllib.request
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.utils.data as data_utils
from torch.optim import SGD, Adam, Adagrad, Adadelta

from sklearn.datasets import load_svmlight_file

import matplotlib.pyplot as plt

import experiments.loss_functions as lf
from experiments.utils import get_dataset

import optuna

torch.set_default_dtype(torch.float64)

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [1135]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [1127]:
def desc_mx(X, y):
    print(f"{X.shape=} \n"
          f"{X.min()=} \n"
          f"{X.max()=} \n"
          f"{y.unique()=} \n"
          f"{torch.linalg.cond(X)=} \n"
          f"Sparsity={1.0 - torch.count_nonzero(X) / X.numel()} \n\n"
          )

In [1105]:
torch.manual_seed(0)
np.random.seed(0)

dataset_name = "colon-cancer"
batch_size = 16

datasets_path = os.getenv("LIBSVM_DIR")
trainX, trainY = load_svmlight_file(f"{datasets_path}/{dataset_name}")

train_data = torch.tensor(trainX.toarray())
train_target = torch.tensor(trainY)

train_load = TensorDataset(train_data, train_target)
train_dataloader = DataLoader(train_load, batch_size=batch_size, shuffle=False)


scale = 6
r1 = -scale
r2 = scale
scaling_vec = (r1 - r2) * torch.rand(train_data.shape[1]) + r2
scaling_vec = torch.pow(torch.e, scaling_vec)
train_data_scaled = scaling_vec * train_data

train_load_scaled = TensorDataset(train_data_scaled, train_target)
train_dataloader_scaled = DataLoader(train_load_scaled, batch_size=batch_size, shuffle=False)

train = train_data, train_target, train_dataloader
train_scaled = train_data_scaled, train_target, train_dataloader_scaled

loss_function = lf.logreg
loss_grad = lf.grad_logreg
loss_hessian = lf.hess_logreg


if loss_function == lf.logreg:
    train_target[train_target == train_target.unique()[0]] = torch.tensor(-1.0, dtype=torch.get_default_dtype())
    train_target[train_target == train_target.unique()[1]] = torch.tensor(1.0, dtype=torch.get_default_dtype())
    assert torch.equal(train_target.unique(), torch.tensor([-1.0, 1.0]))

elif loss_function == lf.nllsq:
    train_target[train_target == train_target.unique()[0]] = 0.0
    train_target[train_target == train_target.unique()[1]] = 1.0
    assert torch.equal(train_target.unique(), torch.tensor([0.0, 1.0]))


desc_mx(train_data, train_target)
desc_mx(train_data_scaled, train_target)
print(loss_function.__name__)

X.shape=torch.Size([62, 2000]) 
X.min()=tensor(-5.2417) 
X.max()=tensor(4.5011) 
y.unique()=tensor([-1.,  1.]) 
torch.linalg.cond(X)=tensor(1935302.8878) 
Sparsity=0.0 


X.shape=torch.Size([62, 2000]) 
X.min()=tensor(-1347.5415) 
X.max()=tensor(1559.9773) 
y.unique()=tensor([-1.,  1.]) 
torch.linalg.cond(X)=tensor(2493442.3704) 
Sparsity=0.0 


logreg


In [1128]:
torch.manual_seed(0)
np.random.seed(0)

n = 1000
d = 1000

train_data = np.random.randn(n, d)
train_data = (train_data + train_data.T) / 2
w_star = np.random.randn(d)

batch_size = 200
EPOCHS = 100

dataset_name = f"synthetic-classification-{n}x{d}"

train_target = train_data @ w_star
train_target[train_target < 0.0] = -1.0
train_target[train_target > 0.0] = 1.0

train_data = torch.Tensor(train_data)
train_target = torch.Tensor(train_target)

train_load = TensorDataset(train_data, train_target)
train_dataloader = DataLoader(train_load, batch_size=batch_size, shuffle=False)

scale = 6
r1 = -scale
r2 = scale
scaling_vec = (r1 - r2) * torch.rand(train_data.shape[1]) + r2
scaling_vec = torch.pow(torch.e, scaling_vec)
train_data_scaled = scaling_vec * train_data

train_load_scaled = TensorDataset(train_data_scaled, train_target)
train_dataloader_scaled = DataLoader(train_load_scaled, batch_size=batch_size, shuffle=False)

train = train_data, train_target, train_dataloader
train_scaled = train_data_scaled, train_target, train_dataloader_scaled

loss_function = lf.logreg
# loss_grad = lf.grad_logreg
# loss_hessian = lf.hess_logreg


if loss_function in [lf.logreg]:
    train_target[train_target == train_target.unique()[0]] = torch.tensor(-1.0, dtype=torch.get_default_dtype())
    train_target[train_target == train_target.unique()[1]] = torch.tensor(1.0, dtype=torch.get_default_dtype())
    assert torch.equal(train_target.unique(), torch.tensor([-1.0, 1.0]))

elif loss_function == lf.nllsq:
    train_target[train_target == train_target.unique()[0]] = 0.0
    train_target[train_target == train_target.unique()[1]] = 1.0
    assert torch.equal(train_target.unique(), torch.tensor([0.0, 1.0]))

desc_mx(train_data, train_target)
desc_mx(train_data_scaled, train_target)
print(loss_function.__name__)

X.shape=torch.Size([1000, 1000]) 
X.min()=tensor(-3.2768) 
X.max()=tensor(3.4887) 
y.unique()=tensor([-1.,  1.]) 
torch.linalg.cond(X)=tensor(2531.8414) 
Sparsity=0.0 


X.shape=torch.Size([1000, 1000]) 
X.min()=tensor(-1250.2118) 
X.max()=tensor(1108.5549) 
y.unique()=tensor([-1.,  1.]) 
torch.linalg.cond(X)=tensor(50207964.6006) 
Sparsity=0.0 


logreg


In [1132]:
opt = Adam
epochs = 500

for dataset, sc in zip([train, train_scaled], [0, scale]):

    def objective(trial):
        torch.manual_seed(0)

        data, target, dataloader = dataset

        # parameters
        w = torch.zeros(data.shape[1], device=device).requires_grad_()
        lr = trial.suggest_float("lr", 1e-20, 10)
        opt = Adam([w], lr=lr)
        
        def compute_loss(w, data, target):
            loss = loss_function(w, data, target)
            loss.backward()
            return loss
        
        for epoch in range(500):
            for i, (batch_data, batch_target) in enumerate(dataloader):
                batch_data = batch_data.to(device)
                batch_target = batch_target.to(device)
                opt.zero_grad()
                loss = compute_loss(w, batch_data, batch_target)
                opt.step()

        loss = loss_function(w, data.to(device), target.to(device))
        return loss

    study_name = f"{dataset_name}-{scale}-{epochs}-{batch_size}-{opt.__name__.lower()}-{loss_function.__name__}"
    storage_name = f"sqlite:///experiments/optuna/{study_name}.db"
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
    study.optimize(objective, n_trials=100)

[I 2023-11-28 13:56:39,315] A new study created in RDB with name: synthetic-classification-1000x1000-0-500-200-adam-logreg
[I 2023-11-28 13:56:44,130] Trial 0 finished with value: 1.91866106159262e-08 and parameters: {'lr': 1.2617714714091588}. Best is trial 0 with value: 1.91866106159262e-08.
[I 2023-11-28 13:56:47,774] Trial 1 finished with value: 1.6929425879877127e-09 and parameters: {'lr': 2.2111502100130154}. Best is trial 1 with value: 1.6929425879877127e-09.
[W 2023-11-28 13:56:51,272] Trial 2 failed with parameters: {'lr': 8.100353772778924} because of the following error: The value nan is not acceptable.
[W 2023-11-28 13:56:51,273] Trial 2 failed with value tensor(nan, grad_fn=<MeanBackward0>).
[W 2023-11-28 13:56:54,818] Trial 3 failed with parameters: {'lr': 7.9770442083073325} because of the following error: The value nan is not acceptable.
[W 2023-11-28 13:56:54,819] Trial 3 failed with value tensor(nan, grad_fn=<MeanBackward0>).
[W 2023-11-28 13:56:58,328] Trial 4 failed

In [ ]:
!pwd

In [1142]:
dataset_name = "leu"
scale = 6
epochs = 500
batch_size = 16
opt = "adagrad"
loss = "logreg"

study_name = f"{dataset_name}-{scale}-{epochs}-{batch_size}-{opt}-{loss}"
storage_name = f"sqlite:///experiments/optuna/{study_name}.db"
study = optuna.load_study(study_name=study_name, storage=storage_name)
study.best_params, study.best_value

({'lr': 0.00999990153926641}, 8.291517751717862e-11)